In [1]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml(
    'mnist_784',
    version=1,
    return_X_y=True
)
X = X.values
y = y.astype(int).values

print(X.shape)
print(y.shape)

In [2]:
# Normalize the pixels values to the range -1 to 1 (instead of 0 to 255)

X = ((X / 255.) - .5) * 2

In [3]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(
    nrows=2,
    ncols=5,
    sharex=True,
    sharey=True
)

ax = ax.flatten()
for i in range(10):
    img = X[y == i][0].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

In [4]:
fig, ax = plt.subplots(
    nrows=5,
    ncols=5,
    sharex=True,
    sharey=True
)
ax = ax.flatten()

for i in range(25):
    img = X[y == 7][i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys')
ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

In [5]:
from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=10000,
    random_state=123,
    stratify=y
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_temp, y_temp,
    test_size=5000,
    random_state=123,
    stratify=y_temp
)

In [6]:
from c11_nn_from_scratch.neuralnet import NeuralNetMLP

model = NeuralNetMLP(
    num_features=28 * 28,
    num_hidden=50,
    num_classes=10
)

In [7]:
from c11_nn_from_scratch.neuralnet import minibatch_generator, MINIBATCH_SIZE, NUM_EPOCHS

for i in range(NUM_EPOCHS):
    minibatch_gen = minibatch_generator(
        X_train, y_train, MINIBATCH_SIZE
    )
    for X_train_mini, y_train_mini in minibatch_gen:
        break
    break

print(X_train_mini.shape)
print(y_train_mini.shape)

In [8]:
import numpy as np
from c11_nn_from_scratch.neuralnet import mse_loss, accuracy

_, probas = model.forward(X_valid)
mse = mse_loss(y_valid, probas)
print(f'Initial validation MSE: {mse:.1f}')

predicted_labels = np.argmax(probas, axis=1)
acc = accuracy(y_valid, predicted_labels)
print(f'Initial validation accuracy: {acc * 100:.1f}%')

In [9]:
from c11_nn_from_scratch.neuralnet import compute_mse_and_acc

mse, acc = compute_mse_and_acc(model, X_valid, y_valid)
print(f'Initial valid MSE: {mse:.1f}')
print(f'Initial valid accuracy: {acc * 100:.1f}%')

In [10]:
from c11_nn_from_scratch.neuralnet import train

np.random.seed(123)

epoch_loss, epoch_train_acc, epoch_valid_acc = train(
    model, X_train, y_train, X_valid, y_valid, num_epochs=50, learning_rate=0.1
)

In [ ]:
plt.plot(range(len(epoch_loss)), epoch_loss)
plt.ylabel('Mean squared error')
plt.xlabel('Epoch')
plt.show()

In [ ]:
plt.plot(range(len(epoch_train_acc)), epoch_train_acc, label='Training')
plt.plot(range(len(epoch_valid_acc)), epoch_valid_acc, label='Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(loc='lower right')
plt.show()

In [ ]:
test_mse, test_acc = compute_mse_and_acc(model, X_test, y_test)
print(f'Test accuracy: {test_acc*100:.2f}%')

In [ ]:
# Misclassified

X_test_subset = X_test[:1000, :]
y_test_subset = y_test[:1000]

_, probas = model.forward(X_test_subset)
test_pred = np.argmax(probas, axis=1)

misclassified_images = X_test_subset[y_test_subset != test_pred][:25]
misclassified_labels = test_pred[y_test_subset != test_pred][:25]
correct_labels = y_test_subset[y_test_subset != test_pred][:25]

fig, ax = plt.subplots(
    nrows=5,
    ncols=5,
    sharex=True,
    sharey=True,
    figsize=(8, 8)
)
ax = ax.flatten()

for i in range(25):
    img = misclassified_images[i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
    ax[i].set_title(
        f'{i + 1}) '
        f'True: {correct_labels[i]}\n'
        f' Predicted: {misclassified_labels[i]}'
    )
ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()